# Lab 5 - Parcel Feature Extraction

Next, we will illustrate the construction of features related to our main task: finding the relationship between property development and water quality over time.  In a previous lab, you identified lakes for which we have complete information for the years from 2004 to 2015.  In this lab, we will

[Original Data and variable information](https://gisdata.mn.gov/organization/us-mn-state-metrogis?q=Metro+Regional+Parcel+Dataset&sort=score+desc%2C+metadata_modified+desc)

## Problem 1 - Feature construction

**Overview.** Remember that our target output file will have one row per year-lake combination.  To attach property information, we will need to group and aggregate the parcel data to create features for each lake-year combination.  When grouping the data, be sure to maintain the variables needed to join to the water quality data, namely the lake ID and year.  Since we are looking at tracking property development/change over time, we will want to generate features tracking

* Number of properties close to each lake,
* Summaries of the value of properties close to each lake,
* Aggregations on the size and type of the properties, and
* Other features that might impact water quality.
    
#### Task 1. Understanding parcel variables

Before we can construct features, we need to make sure we understand the parcel data.  The metro parcel data is provided by the State of Minnesota and the meta data can be found online.  For example, searching for *metro parcel 2014* lead to [this site](https://geo.btaa.org/catalog/304cf3d8-a53b-4ea9-b02a-f550bd68e320).  Clicking on the *Meta data* button in the top left, brought up more information.  Clicking *Download* opened in this meta data [in a separate page](https://resources.gisdata.mn.gov/pub/gdrs/data/pub/us_mn_state_metrogis/plan_regonal_parcels_2014/metadata/metadata.html)

Look through the **Section 4: Attributes** and identify variables that might impact the water quality of near-by lakes.

> <font color="orange"> Your thoughts here </font>


ACRES_POLY: Polygon Acreage

HOMESTEAD: Homestead Status

EMV_LAND: Est. Market Value - Land
EMV_BLDG: Est. Market Value - Buildings
EMV_TOTAL: Est. Market Value - Total

TAX_CAPAC: Tax Capacity
TOTAL_TAX: Total Tax

TAX_EXEMPT: Tax Exempt Status


DWELL_TYPE: Dwelling Type
HOME_STYLE: Home Style
FIN_SQ_FT: Square Footage

GARAGE: Garage Y/N
GARAGESQFT: Garage Square Footage

BASEMENT: Basement Y/N

HEATING: Heating type
COOLING: Cooling type


### Task 2. Feature Brainstorming

Our objective is to build a feature table with one row per lake-year, using grouped summary statistics. Here are effective strategies for feature construction:

1. **Numerical summaries:** Calculate group-level statistics (mean, median, standard deviation, IQR, etc.) for numeric variables.
2. **Categorical summaries:** For text data, consider:
   - **Success rates:** Compute proportions for binary variables (e.g., percent of homes with basements).
   - **Label cleaning:** Review and standardize unique labels to remove duplicates or inconsistencies.
   - **Broader categories:** Recode variables with many rare categories into a smaller, more meaningful set.
   - **Indicator columns:** Create indicator variables and aggregate them to show presence/absence or proportions (e.g., count of each property use type).

Review the variables you identified earlier and outline a feature construction strategy for each.

> <font color="orange"> Your thoughts here </font>

Numeric : 
FIN_SQ_FT

ACRES_POLY

EMV_TOTAL

TOTAL_TAX


Categorical:
 HOMESTEAD: Homestead Status
* TAX_EXEMPT: Tax Exempt Status 
* HEATING: Heating type
* COOLING: Cooling type





### Task 4. Initial querying with filter and select

First, you should build a query that filters the parcel data to 
1. only include parcels within 1600 feet of the lakes we are studying, and 
2. only for the lakes with complete information.  

You should also select only the columns you will need for feature construction and joining to the water quality data.

In [40]:
# your query here
import polars as pl

parcels = (
    pl.scan_parquet("data/parcel_combined.parquet")
    .filter(
        pl.col("distance_category").is_in([
            "within_500m",
            "between_501-1600m"
        ])
    )
)

## Problem 2.  Numerical Summaries

Two important categories of property data involve the size (e.g., finished square footage) and value (e.g., accessed value and/or taxes paid).

**Tasks.** 

1. Identify 2-3 variables for each of these categories.
2. Write a query that computes the summary statistics for each of these variables for each lake-year.  
3. Write this summary table out to a CSV file named `parcel_numerical_summaries.csv`.  Again, you should partition by lake ID and year.

In [26]:
(numerical_summaries := 
    parcels
    .with_columns([
        pl.col("FIN_SQ_FT").cast(pl.Float64),
        pl.col("ACRES_POLY").cast(pl.Float64),
        pl.col("EMV_TOTAL").cast(pl.Float64),
        pl.col("TOTAL_TAX").cast(pl.Float64),
    ])
    .group_by(["Monit_MAP_CODE1", "Year"])
    .agg([
        pl.col("FIN_SQ_FT").mean().alias("mean_FIN_SQ_FT"),
        pl.col("FIN_SQ_FT").median().alias("median_FIN_SQ_FT"),

        pl.col("ACRES_POLY").mean().alias("mean_acres"),
        pl.col("ACRES_POLY").median().alias("median_acres"),

        pl.col("EMV_TOTAL").mean().alias("mean_EMV_TOTAL"),
        pl.col("EMV_TOTAL").median().alias("median_EMV_TOTAL"),

        pl.col("TOTAL_TAX").mean().alias("mean_total_tax"),
        pl.col("TOTAL_TAX").median().alias("median_total_tax"),
    ]).collect()
    .with_columns(
    pl.col("Year").cast(pl.Int32)
))

Monit_MAP_CODE1,Year,mean_FIN_SQ_FT,median_FIN_SQ_FT,mean_acres,median_acres,mean_EMV_TOTAL,median_EMV_TOTAL,mean_total_tax,median_total_tax
str,i32,f64,f64,f64,f64,f64,f64,f64,f64
"""27005300-01""",2011,936.439502,1069.0,0.952865,0.22,254793.498494,195600.0,3875.655762,2568.5
"""10007000-01""",2013,1304.009452,1392.0,5.042382,0.32,224001.512287,167600.0,2091.330813,1662.0
"""10022600-01""",2013,1597.522161,1281.0,1.278234,0.2,250780.193906,143500.0,3801.045706,1996.0
"""02000400-01""",2013,1277.936383,1092.0,3.348357,0.37,175163.660131,160100.0,2933.670588,2821.0
"""13005400-01""",2015,1631.666667,2420.0,40.05,2.55,313100.0,306200.0,0.0,0.0
…,…,…,…,…,…,…,…,…,…
"""10000700-01""",2008,1903.409018,1860.0,0.904167,0.46,426868.1562,350950.0,4558.376812,3724.0
"""27011101-01""",2005,0.0,0.0,0.0,0.0,292870.47619,223300.0,4891.27483,2625.0
"""27001500-01""",2004,0.0,0.0,0.0,0.0,389744.772359,251000.0,7043.045528,2912.0


## Problem 3.  Simple categorical summaries.

In this part, you will create summary statistics for some of the simpler categorical variables.

**Binary variables.** There are two examples of binary variables, listed below.  You will need to compute the percent of `Yes` for each.

* GARAGE: Garage Y/N
* BASEMENT: Basement Y/N

**Other categorical variables.** There are a number of other categorical variables.  You need to select one of these variables, inspect/clean your variable as needed, create indicator variables for each resulting label, and compute summary statistics for each label.

* HOMESTEAD: Homestead Status
* TAX_EXEMPT: Tax Exempt Status 
* DWELL_TYPE: Dwelling Type 
* HOME_STYLE: Home Style
* HEATING: Heating type
* COOLING: Cooling type

**Tasks.**
Create a query that

1. Select one binary and two other categorical variables for feature construction,
2. Reads in the parcel data and selects the relevant columns (be sure to keep the lake ID and year),
3. Inspect unique labels and recode/clean as needed,
4. Create a literal column of ones, and
5. Pivot to get the counts of each label per lake-year (do this once per category).

Write this summary table out to a csv file named `parcel_categorical_summaries.csv`.  Again, you should partition by lake ID and year.

In [49]:

(garage_summary := 
    parcels
    .select([
        "Monit_MAP_CODE1",
        "Year",
        "GARAGE"
    ]).with_columns(pl.lit(1).alias("count"))
      .with_columns(
        pl.col("count")
        .sum()
        .over(["Monit_MAP_CODE1", "Year"])
        .alias("total_count")
    )
    .with_columns(
        (pl.col("GARAGE") == "Y")
        .cast(pl.Int32)
        .sum()
        .over(["Monit_MAP_CODE1", "Year"])
        .alias("garage_yes_count")
    )
    .unique(subset=["Monit_MAP_CODE1", "Year"])
    .with_columns(
        (pl.col("garage_yes_count") / pl.col("total_count") * 100)
        .alias("pct_garage_yes")
    )
    .select([
        "Monit_MAP_CODE1",
        "Year",
        "garage_yes_count",
        "total_count",
        "pct_garage_yes"
    ])
    .with_columns(
           pl.col("Year").cast(pl.Int32)
).collect()
.select(['Monit_MAP_CODE1', 'Year', 'pct_garage_yes']))



Monit_MAP_CODE1,Year,pct_garage_yes
str,i32,f64
"""27008500-01""",2015,76.724715
"""27011800-02""",2004,0.0
"""10006600-01""",2007,26.5625
"""19002800-01""",2008,0.0
"""27008600-01""",2010,77.575377
…,…,…
"""27104501-01""",2007,0.0
"""82010800-01""",2014,68.292683
"""82003000-01""",2007,0.0


In [43]:
(cooling_summary := 
    parcels
    .with_columns(
        pl.when(pl.col("COOLING").is_in([
            "A/CON", "1 AC UNIT", "2 AC UNITS", "3 AC UNITS", "4 AC UNITS",
            "Refrigerated Air using Heatin*", "WALL MOUNT", "RAD/BBELEC"
        ])).then(pl.lit("Air_Cooling"))
        .when(pl.col("COOLING").is_in([
            "CENTRAL W/AIR COND", "CENTRAL", "CENTW/AIR", "CEN.REFRIG",
            "Package Unit", "PKG RF TOP", "Complete HVAC"
        ])).then(pl.lit("AC_Cooling"))
        .when(pl.col("COOLING").is_in([
            "Forced Air", "Forced Air Furnace", "FORCED AIR"
        ])).then(pl.lit("Central_Cooling"))
        .when(pl.col("COOLING").is_in([
            "CEN.EVAP", "Evaporative Cooling", "CHILL WATR"
        ])).then(pl.lit("Other_Cooling"))
        .otherwise(pl.lit("No_Cooling"))
        .alias("COOLING_GROUP")
    )
    .group_by(["Monit_MAP_CODE1", 
               "Year", 
               "COOLING_GROUP"])
    .agg(pl.count().alias("count"))
    .collect()
    .pivot(
        index=["Monit_MAP_CODE1", "Year"],
        columns="COOLING_GROUP",
        values="count"
    )
    .fill_null(0)

)



C:\Users\ym5765bo\AppData\Local\Temp\ipykernel_27428\3393511025.py:24: DeprecationWarning: `pl.count()` is deprecated. Please use `pl.len()` instead.
(Deprecated in version 0.20.5)
  .agg(pl.count().alias("count"))
C:\Users\ym5765bo\AppData\Local\Temp\ipykernel_27428\3393511025.py:26: DeprecationWarning: the argument `columns` for `DataFrame.pivot` is deprecated. It was renamed to `on` in version 1.0.0.
  .pivot(


Monit_MAP_CODE1,Year,Central_Cooling,No_Cooling,AC_Cooling,Other_Cooling,Air_Cooling
str,str,u32,u32,u32,u32,u32
"""27002900-01""","""2011""",1630,4532,0,0,0
"""82009200-01""","""2012""",0,1246,0,0,0
"""10011000-01""","""2007""",0,691,4,50,624
"""27005800-01""","""2013""",1090,5270,0,0,0
"""82048800-01""","""2013""",0,300,0,0,0
…,…,…,…,…,…,…
"""02000700-01""","""2012""",0,1875,0,0,0
"""82030800-01""","""2010""",0,3776,0,0,0
"""82004900-01""","""2014""",0,686,0,0,0


In [50]:
(cooling_percent := 
    cooling_summary
    .with_columns(
        pl.sum_horizontal([
            "Air_Cooling",
            "AC_Cooling",
            "Central_Cooling",
            "Other_Cooling",
            "No_Cooling"
        ]).alias("Total")
    )
    .with_columns([
        (pl.col("Air_Cooling") / pl.col("Total") * 100).alias("Percentage_Air_Cooling"),
        (pl.col("AC_Cooling") / pl.col("Total") * 100).alias("Percentage_AC_Cooling"),
        (pl.col("Central_Cooling") / pl.col("Total") * 100).alias("Percentage_Central_Cooling"),
        (pl.col("Other_Cooling") / pl.col("Total") * 100).alias("Percentage_Other_Cooling"),
        (pl.col("No_Cooling") / pl.col("Total") * 100).alias("Percentage_No_Cooling"),
    ])
    .with_columns(
             pl.col("Year").cast(pl.Int32))
    .select(['Monit_MAP_CODE1', 'Year', 'Percentage_Air_Cooling', 'Percentage_AC_Cooling', 'Percentage_Central_Cooling', 'Percentage_Other_Cooling', 'Percentage_No_Cooling'])
)

Monit_MAP_CODE1,Year,Percentage_Air_Cooling,Percentage_AC_Cooling,Percentage_Central_Cooling,Percentage_Other_Cooling,Percentage_No_Cooling
str,i32,f64,f64,f64,f64,f64
"""27002900-01""",2011,0.0,0.0,26.452451,0.0,73.547549
"""82009200-01""",2012,0.0,0.0,0.0,0.0,100.0
"""10011000-01""",2007,45.580716,0.292184,0.0,3.652301,50.474799
"""27005800-01""",2013,0.0,0.0,17.138365,0.0,82.861635
"""82048800-01""",2013,0.0,0.0,0.0,0.0,100.0
…,…,…,…,…,…,…
"""02000700-01""",2012,0.0,0.0,0.0,0.0,100.0
"""82030800-01""",2010,0.0,0.0,0.0,0.0,100.0
"""82004900-01""",2014,0.0,0.0,0.0,0.0,100.0


In [53]:
(tax_exempt_summary := 
    parcels
    .select([
        "Monit_MAP_CODE1",
        "Year",
        "TAX_EXEMPT"
    ])
     .with_columns(pl.lit(1).alias("count"))
      .group_by(["Monit_MAP_CODE1", "Year", "TAX_EXEMPT"])
      .agg(pl.col("count").sum())
      .collect()
      .pivot(
        index=["Monit_MAP_CODE1", "Year"],
        columns="TAX_EXEMPT",
        values="count"
    )
    .with_columns(
            pl.col("Year").cast(pl.Int32))
    .select(['Monit_MAP_CODE1', 'Year', 'N', 'Y'])
    .fill_null(0)
)

C:\Users\ym5765bo\AppData\Local\Temp\ipykernel_27428\830165721.py:12: DeprecationWarning: the argument `columns` for `DataFrame.pivot` is deprecated. It was renamed to `on` in version 1.0.0.
  .pivot(


Monit_MAP_CODE1,Year,N,Y
str,i32,i32,i32
"""82002500-01""",2015,0,0
"""27010400-02""",2013,2985,180
"""82011700-01""",2007,0,0
"""70007400-01""",2008,0,147
"""82011301-01""",2013,0,0
…,…,…,…
"""10001200-01""",2007,0,0
"""82028700-01""",2014,0,0
"""82012600-01""",2013,0,0


In [54]:
(categorical_summary := 
    garage_summary
    .join(cooling_percent, on=["Monit_MAP_CODE1", "Year"], how="left")
    .join(tax_exempt_summary, on=["Monit_MAP_CODE1", "Year"], how="left")
)

Monit_MAP_CODE1,Year,pct_garage_yes,Percentage_Air_Cooling,Percentage_AC_Cooling,Percentage_Central_Cooling,Percentage_Other_Cooling,Percentage_No_Cooling,N,Y
str,i32,f64,f64,f64,f64,f64,f64,i32,i32
"""27008500-01""",2015,76.724715,0.0,0.0,0.0,0.0,100.0,2830,156
"""27011800-02""",2004,0.0,0.0,0.0,0.0,0.0,100.0,2511,103
"""10006600-01""",2007,26.5625,32.03125,0.0,0.0,0.0,67.96875,0,0
"""19002800-01""",2008,0.0,0.0,0.0,0.0,0.0,100.0,1534,35
"""27008600-01""",2010,77.575377,0.0,0.0,62.876884,0.0,37.123116,1543,49
…,…,…,…,…,…,…,…,…,…
"""27104501-01""",2007,0.0,0.0,0.0,0.0,0.0,100.0,4031,203
"""82010800-01""",2014,68.292683,0.0,0.0,0.0,0.0,100.0,0,0
"""82003000-01""",2007,0.0,0.0,0.0,0.0,0.0,100.0,0,0


## Problem 4.  Join all the summaries.

Finally, you need to join all the summaries created above, along with the water quality summaries created in a previous lab, into one overall summary file.  Write the resulting table to a CSV file named `water_quality_and_parcel_summaries_2004_to_2015.csv`.

In [55]:
(water_quality := pl.read_parquet("water_quality_by_year.parquet"))

water_quality = water_quality.rename({
    "DNR_ID_Site_Number": "Monit_MAP_CODE1"
})

## Problem 5.  Put it all together

It is often useful to package all of the data constructions steps together in one convenient place.  Your last task is to

1. Gather all of your data construction code below.
    * You don't need to include exploratory code, e.g., exploring join mismatches; only the code necessary to combine, clean, and write your data.
2. Clean/refactor the code.
3. Be sure to display all important intermediate results.

In [59]:

(overall_summary := 
    water_quality
    .join(numerical_summaries, on=["Monit_MAP_CODE1", "Year"], how="left")
    .join(categorical_summary, on=["Monit_MAP_CODE1", "Year"], how="left")
)



Monit_MAP_CODE1,LAKE_NAME,Year,avg_secchi_depth,avg_total_phosphorus,mean_FIN_SQ_FT,median_FIN_SQ_FT,mean_acres,median_acres,mean_EMV_TOTAL,median_EMV_TOTAL,mean_total_tax,median_total_tax,pct_garage_yes,Percentage_Air_Cooling,Percentage_AC_Cooling,Percentage_Central_Cooling,Percentage_Other_Cooling,Percentage_No_Cooling,N,Y
str,str,i32,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i32,i32
"""19002300-01""","""Farquar Lake""",2011,0.614286,0.0995,2791.427205,2819.0,0.845574,0.34,302089.450956,281700.0,3885.499075,3695.0,0.0,0.0,0.0,0.0,0.0,100.0,3200,42
"""82013700-01""","""Fish Lake""",2008,1.502722,0.0585,992.348133,0.0,3.703259,0.0,371853.279516,322900.0,212298.504541,4650.0,1.816347,44.601413,0.0,0.0,0.0,55.398587,963,28
"""82008700-01""","""Regional Park Lake""",2005,1.785143,0.050071,0.0,0.0,2.544358,0.04,303607.16763,227900.0,2924.462428,2268.0,0.0,0.0,0.0,0.0,0.0,100.0,797,68
"""02000500-01""","""George Watch Lake""",2006,0.728571,0.164286,1652.141119,1120.0,2.204696,0.67,235092.781833,209700.0,3435.78751,1947.0,0.0,0.0,0.0,0.0,0.0,100.0,1129,104
"""27004201-01""","""Twin Lake""",2007,1.296875,0.09125,0.0,0.0,0.508207,0.23,234480.469552,193600.0,3833.909905,2495.0,0.0,0.0,0.0,0.0,0.0,100.0,6629,186
…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…,…
"""82008900-01""","""Markgraf Lake""",2014,0.964333,0.082083,1388.569201,1464.0,0.864665,0.24,376544.834308,207000.0,0.0,0.0,75.568551,0.0,0.0,0.0,0.0,100.0,0,0
"""13005300-01""","""Big Comfort Lake""",2008,1.514286,0.024929,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null,null
"""82011602-01""","""Armstrong Lake""",2007,0.914429,0.077714,0.0,0.0,0.0,0.0,360547.861507,223800.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,100.0,0,0


In [60]:
overall_summary.write_csv("water_quality_and_parcel_summaries_2004_to_2015.csv")